In [61]:
import random

In [62]:
def load_data_setcover(filename):
    f = open(filename, "r")
    first = f.readline().split()
    num_sets = int(first[0])
    num_elems = int(first[1])
    
    sets = []
    costs = []
    for line in f:
        l = line.split()
        aux = []
        costs.append(int(l[0]))
        for i in range(1, len(l)):
            aux.append(int(l[i]))
        sets.append(set(aux))
        
    #print(items)
    return num_sets, num_elems, sets, costs

In [63]:
def initialization(num_sets, pop_init, size):
    population = []
    #for i in range(pop_init):
    #    population.append(random.sample(range(0,num_sets), k = random.randint(2, num_sets/size)))
     
    for i in range(pop_init):
        ind = []
        for i in range(num_sets):
            if random.randint(0,1) == 1:
                ind.append(i)
        population.append(ind)
    return population

In [64]:
def fitness(chromosome, num_elems, sets, costs):
    covered = set()
    penalty = 0
    for s in chromosome:
        covered = covered.union(sets[s])
        penalty += costs[s]
    for i in range(1, num_elems+1):
        if i not in covered:
            penalty += 1000   
    
    return penalty

In [65]:
def evaluate(population, num_elems, sets, costs):
    fitness_scores = []
    for c in population:
        fitness_scores.append(fitness(c, num_elems, sets, costs))
    return fitness_scores

In [66]:
def selection(population, fitness_scores, pop_max):
    selected_idx = []
    fitscores_aux = fitness_scores
    fitscores_aux.sort(reverse=True)
    i = 0
    while i < pop_max and i < len(fitness_scores):
        selected_idx.append(fitness_scores.index(fitscores_aux[i]))
        i+=1
    selected = []
    for s in selected_idx:
        selected.append(population[s])
    return selected

In [67]:
def crossover(population):
    c = 0
    new_population = []
    while c < len(population):
        parent1 = population[c]
        if (c+1 < len(population)):
            parent2 = population[c+1]
            point = random.randint(1, len(parent2)-1)
            #print(point)
            child1 = parent1[0:point] + parent2[point:]
            child2 = parent2[0:point] + parent1[point:]
            new_population.append(child1)
            new_population.append(child2)
            #print(parent1, parent2, child1, child2)
            #print("____")
        c += 2 
    for n in new_population:
        population.append(n)
    return population

In [74]:
def mutation(population, num_sets, mut_prob):
    all_sets = list(range(num_sets))
    for c in population:
        #if random.randrange(2) == 1:
        #    mutate = random.randrange(len(c))
        #    available_sets = list(set(all_sets) - set(c))
        #    new = random.randrange(len(available_sets))
        #    c[mutate] = available_sets[new]
        
        for i in range(len(c)):
            if random.randrange(100) < mut_prob:
                available_sets = list(set(all_sets) - set(c))
                new = random.randrange(len(available_sets))
                c[i] = available_sets[new]
                
    return population

In [75]:
def setcover_genetic(max_iters, filename):
    num_sets, num_elems, sets, costs = load_data_setcover(filename)
    
    size = 10
    pop_init = 10
    pop_max = 50
    mut_prob = 5
    
    population = initialization(num_sets, pop_init, size)
    fitness_scores = evaluate(population, num_elems, sets, costs)
    i = 0
    while (i < max_iters and min(fitness_scores)!=0) :
        print("--Iteration " + str(i) + "--")
        print("Selection:")
        print(population)
        population = selection(population, fitness_scores, pop_max)
        print("_____")

        print("Crossover:")
        print(population)
        population = crossover(population)
        print("_____")

        print("Mutation:")
        print(population)
        population = mutation(population, num_sets, mut_prob)
        print("_____")
        
        print("Eval:")
        print(population)
        fitness_scores = evaluate(population, num_elems, sets, costs)
        print("____________________________________________")

        i+=1
    
    solution = population[fitness_scores.index(min(fitness_scores))]
    total_cost = 0
    covered = set()
    for i in solution:
        total_cost += costs[i]
        covered = covered.union(sets[i])
    return len(covered),total_cost,solution, covered

In [82]:
l, tc, sol, cov = setcover_genetic(20, "05-inputs/set_cover_100.txt")

--Iteration 0--
Selection:
[[3, 5, 6, 7, 8, 9, 11, 16, 17, 18, 19, 20, 21, 22, 24, 26, 31, 32, 36, 38, 39, 43, 44, 48, 49, 50, 51, 52, 55, 56, 61, 64, 66, 67, 68, 69, 72, 74, 75, 76, 77, 78, 79, 82, 84, 87, 88, 89, 96, 97, 99], [1, 2, 3, 5, 7, 17, 18, 19, 20, 24, 25, 26, 29, 30, 31, 37, 38, 39, 40, 41, 42, 46, 49, 54, 55, 56, 57, 59, 60, 71, 72, 73, 75, 76, 77, 79, 80, 83, 84, 85, 87, 88, 96, 98], [0, 1, 2, 3, 4, 11, 12, 13, 14, 17, 18, 19, 23, 33, 35, 38, 43, 44, 46, 48, 49, 52, 53, 54, 55, 56, 57, 58, 60, 61, 62, 64, 65, 67, 68, 69, 70, 71, 72, 73, 74, 75, 77, 78, 79, 82, 86, 90, 91, 92, 93, 96, 97, 99], [2, 4, 5, 6, 7, 8, 11, 12, 13, 15, 18, 20, 23, 24, 25, 27, 28, 31, 32, 34, 36, 39, 40, 41, 43, 45, 47, 49, 52, 53, 58, 59, 60, 62, 66, 69, 71, 74, 75, 78, 80, 83, 85, 86, 92, 94], [1, 2, 7, 8, 10, 11, 12, 13, 14, 15, 16, 17, 19, 20, 23, 25, 27, 31, 32, 38, 40, 43, 44, 45, 52, 55, 57, 58, 59, 60, 61, 66, 67, 70, 71, 72, 73, 74, 76, 77, 78, 80, 82, 83, 85, 87, 91, 92, 93, 94, 96, 99], 

____________________________________________
--Iteration 6--
Selection:
[[3, 5, 6, 7, 8, 9, 11, 16, 17, 18, 81, 20, 21, 22, 24, 26, 31, 32, 79, 38, 39, 43, 44, 48, 49, 19, 51, 52, 42, 56, 61, 64, 14, 67, 68, 69, 27, 74, 12, 76, 77, 58, 37, 80, 4, 46, 88, 89, 91, 97, 99], [1, 2, 3, 12, 7, 17, 18, 19, 20, 24, 25, 26, 29, 30, 31, 91, 38, 58, 40, 41, 42, 46, 49, 54, 32, 97, 57, 59, 60, 71, 61, 73, 75, 76, 77, 79, 80, 65, 84, 85, 87, 78, 96, 98], [95, 1, 26, 3, 4, 11, 12, 13, 14, 17, 87, 19, 23, 33, 35, 49, 94, 44, 46, 48, 41, 52, 80, 88, 34, 56, 57, 58, 60, 61, 62, 64, 65, 32, 68, 69, 45, 71, 72, 37, 74, 75, 77, 24, 79, 82, 86, 90, 63, 83, 93, 96, 97, 89], [2, 4, 5, 83, 50, 8, 77, 12, 13, 15, 57, 20, 23, 24, 25, 27, 26, 31, 32, 34, 36, 39, 40, 46, 43, 56, 18, 49, 30, 53, 58, 59, 72, 1, 66, 55, 71, 35, 63, 78, 14, 91, 85, 86, 92, 94], [1, 2, 34, 81, 10, 11, 33, 13, 3, 39, 16, 58, 53, 20, 23, 25, 86, 31, 97, 38, 40, 43, 69, 45, 4, 46, 57, 0, 37, 65, 68, 66, 67, 70, 71, 72, 73, 89, 63, 77, 78

____________________________________________
--Iteration 10--
Selection:
[[3, 5, 6, 40, 8, 9, 11, 71, 17, 18, 81, 23, 21, 10, 24, 26, 31, 32, 79, 38, 39, 43, 96, 85, 49, 41, 51, 52, 54, 56, 50, 64, 14, 67, 68, 69, 62, 74, 12, 76, 55, 58, 37, 80, 4, 90, 46, 89, 60, 97, 99], [1, 2, 3, 12, 7, 17, 18, 19, 20, 24, 25, 26, 69, 30, 31, 91, 38, 58, 40, 41, 42, 46, 49, 54, 32, 97, 57, 59, 62, 71, 61, 73, 75, 76, 77, 79, 80, 48, 84, 85, 53, 78, 96, 98], [95, 1, 26, 3, 4, 11, 12, 13, 14, 17, 87, 28, 23, 33, 35, 49, 94, 44, 73, 48, 41, 52, 80, 88, 34, 22, 57, 58, 60, 61, 62, 64, 42, 32, 68, 69, 45, 71, 29, 37, 74, 75, 77, 24, 76, 82, 86, 90, 63, 83, 93, 96, 97, 89], [2, 4, 5, 83, 50, 8, 77, 70, 13, 15, 57, 20, 23, 33, 25, 27, 26, 31, 32, 34, 36, 39, 40, 45, 43, 21, 18, 49, 30, 53, 58, 59, 72, 1, 66, 74, 71, 35, 63, 78, 14, 91, 85, 86, 92, 94], [91, 2, 34, 81, 10, 7, 33, 13, 3, 41, 16, 58, 53, 20, 23, 60, 86, 31, 97, 38, 40, 54, 69, 45, 4, 46, 57, 0, 37, 65, 68, 66, 67, 70, 71, 62, 73, 89, 30, 77, 

[[3, 5, 6, 40, 8, 9, 11, 71, 17, 18, 45, 23, 21, 10, 24, 26, 31, 32, 79, 38, 39, 43, 96, 85, 49, 19, 25, 52, 54, 56, 50, 64, 60, 66, 68, 61, 62, 74, 12, 30, 93, 58, 37, 80, 4, 90, 94, 89, 2, 97, 99], [1, 2, 3, 12, 7, 17, 18, 28, 20, 24, 25, 26, 69, 30, 22, 91, 38, 58, 40, 64, 42, 46, 49, 54, 32, 47, 57, 59, 44, 71, 61, 81, 75, 76, 77, 79, 80, 48, 84, 85, 53, 78, 96, 98], [95, 1, 26, 3, 4, 11, 81, 13, 14, 17, 40, 53, 47, 33, 35, 49, 7, 44, 73, 25, 41, 52, 80, 88, 67, 22, 57, 10, 60, 61, 62, 64, 42, 32, 68, 69, 45, 71, 29, 34, 74, 75, 36, 21, 76, 82, 86, 38, 63, 83, 93, 96, 97, 89], [2, 98, 5, 83, 50, 8, 77, 70, 13, 15, 57, 20, 23, 33, 25, 27, 99, 31, 32, 34, 36, 39, 40, 7, 43, 21, 18, 49, 30, 53, 58, 59, 72, 1, 66, 74, 71, 35, 63, 78, 14, 91, 85, 86, 92, 94], [91, 2, 34, 81, 10, 7, 33, 13, 3, 41, 16, 58, 53, 20, 23, 60, 86, 31, 97, 38, 40, 54, 69, 45, 4, 46, 57, 0, 37, 76, 68, 99, 19, 70, 71, 62, 73, 25, 30, 42, 78, 92, 82, 83, 85, 52, 75, 9, 93, 94, 96, 51], [5, 7, 26, 10, 11, 12, 70, 

____________________________________________
--Iteration 17--
Selection:
[[3, 7, 6, 40, 8, 9, 11, 71, 17, 18, 45, 23, 48, 10, 24, 26, 31, 88, 79, 38, 39, 43, 96, 53, 49, 19, 25, 52, 54, 56, 50, 64, 60, 66, 68, 77, 62, 74, 12, 30, 93, 58, 37, 47, 85, 90, 94, 89, 2, 97, 99], [1, 50, 3, 82, 7, 17, 18, 34, 14, 24, 90, 26, 69, 39, 22, 91, 38, 58, 40, 64, 42, 46, 49, 6, 32, 47, 57, 52, 44, 99, 61, 81, 75, 93, 28, 79, 80, 48, 11, 85, 53, 78, 96, 98], [95, 1, 72, 3, 4, 11, 81, 37, 14, 69, 40, 66, 47, 33, 35, 49, 55, 44, 73, 70, 41, 52, 80, 88, 67, 22, 57, 10, 0, 61, 62, 99, 42, 32, 68, 2, 30, 71, 29, 34, 17, 75, 36, 21, 76, 82, 86, 77, 63, 83, 93, 96, 97, 89], [2, 26, 5, 83, 50, 8, 77, 70, 98, 15, 57, 20, 23, 33, 25, 27, 99, 31, 32, 34, 36, 39, 60, 7, 43, 21, 18, 49, 93, 53, 58, 28, 72, 1, 66, 74, 71, 35, 63, 78, 14, 91, 85, 86, 92, 94], [5, 2, 34, 81, 10, 7, 33, 61, 3, 41, 16, 58, 53, 27, 23, 60, 86, 31, 97, 90, 66, 54, 69, 45, 4, 29, 57, 0, 37, 76, 68, 99, 19, 59, 71, 62, 73, 25, 30, 42, 78,

In [83]:
print(l, tc)

970 2856
